In [1]:
%matplotlib inline
# Dependencies
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statistics
import gmaps
import requests
# Import the API key.
from config import g_key

# Load in csv
vacation_search_df = pd.read_csv("WeatherPy_Challenge_Data/WeatherPy_database.csv")
vacation_search_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Akonolinga,CM,3.77,12.25,67.66,98,100,2.95,light rain,0.88,0.0
1,1,Nortelandia,BR,-14.45,-56.80,67.46,92,0,2.89,clear sky,0.00,0.0
2,2,Russell,US,32.35,-85.20,86.00,79,75,1.72,broken clouds,0.00,0.0
3,3,San Pedro,PH,14.35,121.02,82.40,88,90,3.36,overcast clouds,0.00,0.0
4,4,Alta Floresta,BR,-9.88,-56.09,73.26,58,0,2.73,clear sky,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
579,579,Weligama,LK,5.97,80.42,80.10,84,87,9.06,overcast clouds,0.00,0.0
580,580,Jabinyanah,TN,35.03,10.91,80.60,74,20,3.36,few clouds,0.00,0.0
581,581,Madingou,CG,-4.15,13.55,69.55,80,0,2.73,clear sky,0.00,0.0
582,582,Vila,PT,42.03,-8.16,57.99,94,53,1.72,broken clouds,0.00,0.0


In [2]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain = input("Do you want it to be raining? (yes/no) ")
snow = input("Do you want it to be snowing? (yes/no) ")

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [3]:
if rain == "no" and snow == "no":
    # Filter the dataset to find the cities that fit the criteria.
    preferred_cities_df = vacation_search_df.loc[(vacation_search_df["Max Temp"] <= max_temp) & \
                                                 (vacation_search_df["Max Temp"] >= min_temp) & \
                                                 (vacation_search_df["Rain (inches)"] == 0) & \
                                                 (vacation_search_df["Snow (inches)"] == 0)]
elif rain == "yes" and snow == "no":
    # Filter the dataset to find the cities that fit the criteria.
    preferred_cities_df = vacation_search_df.loc[(vacation_search_df["Max Temp"] <= max_temp) & \
                                                 (vacation_search_df["Max Temp"] >= min_temp) & \
                                                 (vacation_search_df["Rain (inches)"] > 0) & \
                                                 (vacation_search_df["Snow (inches)"] == 0)]

elif rain == "no" and snow == "yes":
    # Filter the dataset to find the cities that fit the criteria.
    preferred_cities_df = vacation_search_df.loc[(vacation_search_df["Max Temp"] <= max_temp) & \
                                                 (vacation_search_df["Max Temp"] >= min_temp) & \
                                                 (vacation_search_df["Rain (inches)"] == 0) & \
                                                 (vacation_search_df["Snow (inches)"] > 0)]

else:
# Filter the dataset to find the cities that fit the criteria.
    preferred_cities_df = vacation_search_df.loc[(vacation_search_df["Max Temp"] <= max_temp) & \
                                                 (vacation_search_df["Max Temp"] >= min_temp) & \
                                                 (vacation_search_df["Rain (inches)"] > 0) & \
                                                 (vacation_search_df["Snow (inches)"] > 0)]

preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
2,2,Russell,US,32.35,-85.20,86.00,79,75,1.72,broken clouds,0.0,0.0
3,3,San Pedro,PH,14.35,121.02,82.40,88,90,3.36,overcast clouds,0.0,0.0
5,5,Vila Velha,BR,-20.33,-40.29,75.20,73,0,9.17,clear sky,0.0,0.0
9,9,Vaini,TO,-21.20,-175.20,78.80,83,75,9.17,broken clouds,0.0,0.0
11,11,Adre,TD,13.47,22.20,79.99,44,43,2.82,scattered clouds,0.0,0.0
17,17,Avarua,CK,-21.21,-159.78,75.20,64,0,18.34,clear sky,0.0,0.0
18,18,Tautira,PF,-17.73,-149.15,82.40,61,20,6.93,few clouds,0.0,0.0
19,19,Hilo,US,19.73,-155.09,84.20,62,40,9.17,scattered clouds,0.0,0.0
31,31,Padang,ID,-0.95,100.35,80.96,64,31,2.75,scattered clouds,0.0,0.0
32,32,Butaritari,KI,3.07,172.79,82.80,75,31,13.98,scattered clouds,0.0,0.0


In [4]:
# Create a DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Russell,US,86.00,broken clouds,32.35,-85.20,
3,San Pedro,PH,82.40,overcast clouds,14.35,121.02,
5,Vila Velha,BR,75.20,clear sky,-20.33,-40.29,
9,Vaini,TO,78.80,broken clouds,-21.20,-175.20,
11,Adre,TD,79.99,scattered clouds,13.47,22.20,
17,Avarua,CK,75.20,clear sky,-21.21,-159.78,
18,Tautira,PF,82.40,few clouds,-17.73,-149.15,
19,Hilo,US,84.20,scattered clouds,19.73,-155.09,
31,Padang,ID,80.96,scattered clouds,-0.95,100.35,
32,Butaritari,KI,82.80,scattered clouds,3.07,172.79,


In [5]:
# Set the parameters to search for a hotel.
params = {"radius": 5000, "type": "lodging", "key": g_key}

In [6]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print(hotels)

{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESUL

In [7]:
print(hotel_df["Hotel Name"])

2                        Macs
3      OYO 412 Viewfort Venue
5        Hotel Vitória Palace
9         Keleti Beach Resort
11                           
                ...          
567      Colinas de Matasiete
571               Hotel Marys
573             Super 8 Hotel
579       Weligama Bay Resort
580         maison belmabrouk
Name: Hotel Name, Length: 176, dtype: object


In [11]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [39]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [41]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=True,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [42]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Russell,US,86.00,broken clouds,32.35,-85.20,Macs
3,San Pedro,PH,82.40,overcast clouds,14.35,121.02,OYO 412 Viewfort Venue
5,Vila Velha,BR,75.20,clear sky,-20.33,-40.29,Hotel Vitória Palace
9,Vaini,TO,78.80,broken clouds,-21.20,-175.20,Keleti Beach Resort
11,Adre,TD,79.99,scattered clouds,13.47,22.20,
17,Avarua,CK,75.20,clear sky,-21.21,-159.78,Paradise Inn
18,Tautira,PF,82.40,few clouds,-17.73,-149.15,Pension Oaoa
19,Hilo,US,84.20,scattered clouds,19.73,-155.09,Hilo Hawaiian Hotel
31,Padang,ID,80.96,scattered clouds,-0.95,100.35,Mercure Padang
32,Butaritari,KI,82.80,scattered clouds,3.07,172.79,Isles Sunset Lodge


In [16]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Challenge_Data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")